# Full Stack Feature Implementation: Interests Management

## Purpose of the Program

The purpose of this program is to provide a full-stack solution for managing user interests. Users can add, update, and delete their interests through a web interface, and the changes are reflected in the backend database.

## Purpose of the Feature

The feature allows users to:
- Add new interests
- Update existing interests
- Delete specific interests

## Input/Output Requests

### Demo: Input to Full Stack Feature

#### Using Frontend to Show API Request and Present API Response

```html
<!-- filepath: /home/yashunix/nighthawk/prism/prism_frontend/navigation/authentication/profile.md -->
<script type="module">
import { pythonURI, fetchOptions } from '{{site.baseurl}}/assets/js/api/config.js';

// Function to update user interests
async function updateProfile(field, value) {
    try {
        if (field === 'interests' && value) {
            const response = await fetch(pythonURI + "/api/user", fetchOptions);
            const userData = await response.json();
            const currentInterests = userData.interests ? userData.interests.split(',').map(i => i.trim()) : [];
            const newInterests = value.split(',').map(i => i.trim());
            const combinedInterests = [...new Set([...currentInterests, ...newInterests])];
            value = combinedInterests.join(', ');

            const updateResponse = await fetch(pythonURI + "/api/interests", {
                ...fetchOptions,
                method: 'PUT',
                body: JSON.stringify({ interests: value })
            });

            if (!updateResponse.ok) {
                throw new Error('Failed to update interests');
            }

            showError('Interests updated successfully', 'green');
            updateUserInfo();
            return;
        }
    } catch (error) {
        console.error('Error updating profile:', error);
        showError('Error updating profile');
    }
}

// Attach the updateProfile function to the window object
window.updateProfile = updateProfile;
</script>
```

#### Using Postman to Show Raw API Request and RESTful Response

**POST Request to Add Interests:**
- URL: `http://localhost:5000/api/interests`
- Method: `POST`
- Body:
  ```json
  {
    "interests": "Reading, Coding, Hiking"
  }
  ```

**Response:**
- Status: `200 OK`
- Body:
  ```json
  {
    "message": "Interests added successfully",
    "interests": "Reading, Coding, Hiking"
  }
  ```

**DELETE Request to Remove an Interest:**
- URL: `http://localhost:5000/api/interests`
- Method: `DELETE`
- Body:
  ```json
  {
    "interest": "Coding"
  }
  ```

**Response:**
- Status: `200 OK`
- Body:
  ```json
  {
    "message": "Interest deleted successfully",
    "interests": "Reading, Hiking"
  }
  ```

### Using db_init, db_restore, db_backup to Show Tester Data Creation and Data Recovery

```python
# filepath: /home/yashunix/nighthawk/prism/prism_backend/db_init.py
from __init__ import db
from model.user import User

# Initialize the database
db.create_all()

# Add sample data
user = User(name="John Doe", uid="johndoe", interests="Reading, Coding, Hiking")
db.session.add(user)
db.session.commit()

print("Database initialized with sample data.")
```

## List Requests

### Use of List, Dictionaries, and Database

**Code Description:**

- **Lists:** Used to manage user interests.
- **Dictionaries:** Used to handle JSON data in API requests and responses.
- **Database:** Used to store user data, including interests.

**Formatting Response Data (JSON) from API into DOM:**

```javascript
// filepath: /home/yashunix/nighthawk/prism/prism_frontend/navigation/authentication/profile.md
function createInterestCards(interests) {
    const interestsSection = document.getElementById('interestsSection');
    interestsSection.innerHTML = '';
    
    interests.forEach(interest => {
        const card = document.createElement('div');
        card.className = 'card';
        card.innerHTML = `
            <h4>${interest}</h4>
            <img src="https://placehold.co/300x200/d34e3f/a3adbf/png?text=${interest}" alt="${interest}">
            <button onclick="deleteInterest('${interest}')">Delete</button>
        `;
        interestsSection.appendChild(card);
    });
}
```

**Queries from Database:**

```python
# filepath: /home/yashunix/nighthawk/prism/prism_backend/api/interests.py
@token_required()
def get(self):
    """
    Return the interests of the authenticated user as a JSON object.
    """
    current_user = g.current_user
    interests = current_user.interests
    if not interests:
        return {'message': 'No interests found for this user'}, 404
    return jsonify(interests)
```

**Methods in Class to Work with Columns:**

```python
# filepath: /home/yashunix/nighthawk/prism/prism_backend/api/interests.py
class InterestsAPI:
    class _CRUD(Resource):
        @token_required()
        def post(self):
            """
            Add new interests to the authenticated user.
            """
            current_user = g.current_user
            body = request.get_json()
            new_interests = body.get('interests')
            if not new_interests:
                return {'message': 'No interests provided'}, 400

            current_user.interests = new_interests
            current_user.update()
            return jsonify(current_user.interests)
```

## Algorithmic Code Request

### Definition of Code Blocks to Handle a Request

**API Class to Perform CRUD Methods:**

```python
# filepath: /home/yashunix/nighthawk/prism/prism_backend/api/interests.py
class InterestsAPI:
    class _CRUD(Resource):
        @token_required()
        def post(self):
            """
            Add new interests to the authenticated user.
            """
            current_user = g.current_user
            body = request.get_json()
            new_interests = body.get('interests')
            if not new_interests:
                return {'message': 'No interests provided'}, 400

            current_user.interests = new_interests
            current_user.update()
            return jsonify(current_user.interests)

        @token_required()
        def put(self):
            """
            Update the interests of the authenticated user.
            """
            current_user = g.current_user
            body = request.get_json()
            updated_interests = body.get('interests')
            if not updated_interests:
                return {'message': 'No interests provided'}, 400

            current_user.interests = updated_interests
            current_user.update(body)
            return jsonify(current_user.interests)

        @token_required()
        def delete(self):
            """
            Delete a specified interest of the authenticated user.
            """
            body = request.get_json()

            if not body or 'interest' not in body:
                return {'message': 'No interest provided'}, 400
            
            current_user = g.current_user
            interest_to_delete = body.get('interest')
            interests = current_user.interests.split(', ')

            if interest_to_delete not in interests:
                return {'message': 'Interest not found'}, 404

            interests.remove(interest_to_delete)
            current_user.interests = ', '.join(interests)
            current_user.update({'interests': current_user.interests})

            return {'message': 'Interest deleted successfully'}, 200
```

### Method/Procedure in Class with Sequencing, Selection, and Iteration

**Example:**

```python
# filepath: /home/yashunix/nighthawk/prism/prism_backend/api/interests.py
@token_required()
def delete(self):
    """
    Delete a specified interest of the authenticated user.
    """
    body = request.get_json()

    if not body or 'interest' not in body:
        return {'message': 'No interest provided'}, 400
    
    current_user = g.current_user
    interest_to_delete = body.get('interest')
    interests = current_user.interests.split(', ')

    if interest_to_delete not in interests:
        return {'message': 'Interest not found'}, 404

    interests.remove(interest_to_delete)
    current_user.interests = ', '.join(interests)
    current_user.update({'interests': current_user.interests})

    return {'message': 'Interest deleted successfully'}, 200
```

### Parameters and Return Type

**Parameters:**
- `body`: JSON object containing the interest to be deleted.

**Return Type:**
- JSON response with a message indicating the result of the operation.

## Call to Algorithm Request

### Definition of Code Block to Make a Request

**Frontend Fetch to Endpoint:**

```javascript
// filepath: /home/yashunix/nighthawk/prism/prism_frontend/navigation/authentication/profile.md
async function deleteInterest(interest) {
    try {
        const response = await fetch(pythonURI + "/api/interests", {
            ...fetchOptions,
            method: 'DELETE',
            body: JSON.stringify({ interest: interest })
        });

        if (!response.ok) {
            throw new Error('Failed to delete interest');
        }

        showError('Interest deleted successfully', 'green');
        updateUserInfo();
    } catch (error) {
        console.error('Error deleting interest:', error);
        showError('Error deleting interest');
    }
}

window.deleteInterest = deleteInterest;
```

### Discuss the Call/Request to the Method with Algorithm

**Call/Request:**
- The `deleteInterest` function sends a DELETE request to the `/api/interests` endpoint with the interest to be deleted.

**Return/Response:**
- The response from the backend is handled by checking the status code and updating the UI accordingly.

### Handling Data and Error Conditions

**Normal Conditions:**
- The interest is successfully deleted, and the UI is updated to reflect the change.

**Error Conditions:**
- If the interest is not found or the request fails, an error message is displayed.

## Conclusion

This blog demonstrates how to implement a full-stack feature for managing user interests. The feature includes adding, updating, and deleting interests, with proper handling of input/output requests, database interactions, and error conditions. By following this approach, you can create a robust and user-friendly application that allows users to manage their interests effectively.